<a href="https://colab.research.google.com/github/dattali18/machine_learning_msc_project/blob/research/daniel/notebook/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heart Disease Dataset

## Context

This data set dates from 1988 and consists of four databases: Cleveland, Hungary, Switzerland, and Long Beach V. It contains 76 attributes, including the predicted attribute, but all published experiments refer to using a subset of 14 of them. The "target" field refers to the presence of heart disease in the patient. It is integer valued 0 = no disease and 1 = disease.

## Content
Attribute Information:

1. age
2. sex
3. chest pain type (4 values)
4. resting blood pressure
5. serum cholestoral in mg/dl
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results (values 0,1,2)
8. maximum heart rate achieved
9. exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment
12. number of major vessels (0-3) colored by flourosopy
13. thal: 0 = normal; 1 = fixed defect; 2 = reversable defect

The names and social security numbers of the patients were recently removed from the database, replaced with dummy values.

## Data Preprocessing

In [1]:
DATA_PATH = "https://github.com/dattali18/machine_learning_msc_project/blob/main/db/heart.csv?raw=true"

In [2]:
import pandas as pd

In [3]:
heart_data = pd.read_csv(DATA_PATH)

In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [5]:
full_name_attrs = [
    'age',
    'sex',
    'chest pain type',
    'resting blood pressure',
    'serum colestoral mg/dl',
    'fasting blood sugar > 120 mg/dl',
    'resting electrocardiographic results',
    'maximum heart rate achieved',
    'exercise induced angina',
    'oldpeak = ST depression induced by exercise relative to rest',
    'the slope of the peak exercise ST segment',
    'number of major vessels (0-3) colored by flourosopy',
    'thal: 0 = normal; 1 = fixed defect; 2 = reversable defect'
]

In [6]:
# split the data into 80-20 train-test using  scikit-learn
from sklearn.model_selection import train_test_split

In [7]:
# the target attribute is 'target'
X = heart_data.drop('target', axis=1)
y = heart_data['target']

In [8]:
# print the shape of X, y
X.shape, y.shape

((1025, 13), (1025,))

In this part we will do scaling for the distance based model K-NN, SVM

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
scaler = StandardScaler()

In [11]:
X_scaled = scaler.fit_transform(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True, stratify=None
)

In [13]:
X_train_scaled, X_test_scaled, y_train, y_test =  train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, shuffle=True, stratify=None
)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((820, 13), (205, 13), (820,), (205,))

## Distance Based Models

### K-NN

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

#### Scaled data

In [16]:
knn_simple_scaled = KNeighborsClassifier(n_neighbors=5)

knn_simple_scaled.fit(X_train_scaled, y_train)

y_pred_simple_scaled = knn_simple_scaled.predict(X_test_scaled)

print("--- Simple k-NN (k=5) ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_simple_scaled):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_simple_scaled))

--- Simple k-NN (k=5) ---
Accuracy: 0.8341

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.77      0.82       102
           1       0.80      0.89      0.84       103

    accuracy                           0.83       205
   macro avg       0.84      0.83      0.83       205
weighted avg       0.84      0.83      0.83       205



In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
param_grid = {
    'n_neighbors': list(range(2, 31)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'] # Testing different distance metrics
}

In [19]:
grid_search_scaled = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy', verbose=1)

grid_search_scaled.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 116 candidates, totalling 580 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                         14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                                         24, 25, 26, 27, 28, 29, 30],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=1)

In [20]:
print("\n--- Grid Search Results ---")
print(f"Best Parameters: {grid_search_scaled.best_params_}")
print(f"Best Cross-Validation Score: {grid_search_scaled.best_score_:.4f}")


--- Grid Search Results ---
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 28, 'weights': 'distance'}
Best Cross-Validation Score: 0.9854


In [21]:
best_knn_scaled = grid_search_scaled.best_estimator_
y_pred_best_scaled = best_knn_scaled.predict(X_test_scaled)

print(f"\nFinal Test Accuracy (Optimized): {accuracy_score(y_test, y_pred_best_scaled):.4f}")
print("\nOptimized Classification Report:")
print(classification_report(y_test, y_pred_best_scaled))


Final Test Accuracy (Optimized): 1.0000

Optimized Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       102
           1       1.00      1.00      1.00       103

    accuracy                           1.00       205
   macro avg       1.00      1.00      1.00       205
weighted avg       1.00      1.00      1.00       205



#### Unscaled Data

In [22]:
knn_simple = KNeighborsClassifier(n_neighbors=5)

knn_simple.fit(X_train, y_train)

y_pred_simple = knn_simple.predict(X_test)

print("--- Simple k-NN (k=5) ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_simple):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_simple))

--- Simple k-NN (k=5) ---
Accuracy: 0.7317

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.73      0.73       102
           1       0.73      0.74      0.73       103

    accuracy                           0.73       205
   macro avg       0.73      0.73      0.73       205
weighted avg       0.73      0.73      0.73       205



In [23]:
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy', verbose=1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 116 candidates, totalling 580 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                         14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                                         24, 25, 26, 27, 28, 29, 30],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=1)

In [24]:
print("\n--- Grid Search Results ---")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Score: {grid_search.best_score_:.4f}")


--- Grid Search Results ---
Best Parameters: {'metric': 'euclidean', 'n_neighbors': 2, 'weights': 'distance'}
Best Cross-Validation Score: 0.9720


In [25]:
best_knn = grid_search.best_estimator_
y_pred_best = best_knn.predict(X_test)

print(f"\nFinal Test Accuracy (Optimized): {accuracy_score(y_test, y_pred_best):.4f}")
print("\nOptimized Classification Report:")
print(classification_report(y_test, y_pred_best))


Final Test Accuracy (Optimized): 0.9854

Optimized Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       102
           1       1.00      0.97      0.99       103

    accuracy                           0.99       205
   macro avg       0.99      0.99      0.99       205
weighted avg       0.99      0.99      0.99       205



### SVM

#### Using unscaled data

In [26]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [27]:
svm_simple = SVC(kernel='linear', C=1.0)

svm_simple.fit(X_train, y_train)

SVC(kernel='linear')

In [28]:
y_pred_svm = svm_simple.predict(X_test)

print("--- Simple Linear SVM ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm))

--- Simple Linear SVM ---
Accuracy: 0.8049

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.71      0.78       102
           1       0.76      0.90      0.82       103

    accuracy                           0.80       205
   macro avg       0.82      0.80      0.80       205
weighted avg       0.82      0.80      0.80       205



In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
# param_grid_svm = {
#     'C': [0.1, 1, 10, 100],
#     'gamma': [1, 0.1, 0.01, 0.001],
#     'kernel': ['rbf', 'poly', 'sigmoid', 'linear']
# }

param_grid_svm = {
    'C': [0.1, 1, 10],
    'gamma': [10.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid', 'linear']
}

In [ ]:
grid_svm = GridSearchCV(SVC(), param_grid_svm, refit=True, verbose=1, cv=5)

grid_svm.fit(X_train, y_train)

print("\n--- SVM Grid Search Results ---")
print(f"Best Parameters: {grid_svm.best_params_}")
print(f"Best CV Score: {grid_svm.best_score_:.4f}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [ ]:
best_svm = grid_svm.best_estimator_
y_pred_best_svm = best_svm.predict(X_test)

print(f"\nFinal Test Accuracy (Optimized SVM): {accuracy_score(y_test, y_pred_best_svm):.4f}")
print("\nOptimized SVM Classification Report:")
print(classification_report(y_test, y_pred_best_svm))

#### Scaled Data

In [ ]:
svm_simple_scaled = SVC(kernel='linear', C=1.0)

svm_simple_scaled.fit(X_train_scaled, y_train)

In [ ]:
y_pred_svm = svm_simple_scaled.predict(X_test)

print("--- Simple Linear SVM ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm))

In [ ]:
grid_svm_scaled = GridSearchCV(SVC(), param_grid_svm, refit=True, verbose=1, cv=5)

grid_svm_scaled.fit(X_train_scaled, y_train)

print("\n--- SVM Grid Search Results ---")
print(f"Best Parameters: {grid_svm.best_params_}")
print(f"Best CV Score: {grid_svm.best_score_:.4f}")

In [ ]:
best_svm = grid_svm_scaled.best_estimator_
y_pred_best_svm = grid_svm_scaled.predict(X_test)

print(f"\nFinal Test Accuracy (Optimized SVM): {accuracy_score(y_test, y_pred_best_svm):.4f}")
print("\nOptimized SVM Classification Report:")
print(classification_report(y_test, y_pred_best_svm))